In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time

In [2]:
def store_results_dict(results, name):
    pickle.dump(results, open("./Results/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "./Results/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### Data Generation

#### Raw Data

In [3]:
# GLOBAL CONFIG

# Var count
n_vars = 10

# Data types (default is standard normal)
binary_indeces = [1, 3, 6, 8, 9]
binarize = True

# Associations between vars an treat/outcome
treat_vars = [0,1,2,3,4,5,6,7]
outcome_vars = [0,1,2,3,4,8,9,10]

# Treat/outcome generation weights
assignment_weights = np.array([0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7])
outcome_weights = np.array([-3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26])
true_treat_effect = -0.4

def generate_data(n_samples=1000):
    # Generate 10 Random Vars
    # 1-4 are confounders: associated with outcome + treatment
    # 5-7 are exposure predictors
    # 8-10 are outcome predictors
    X = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_vars))

    # Binarize specified vars if requested.
    if binarize:
        for var in binary_indeces:
            X[:, var-1] = (X[:, var -1] > 0).astype(int)

    # Add dummy for bias param     
    X = np.hstack([np.ones((n_samples, 1)), X])
    return X

In [4]:
# DEBUG
# X = generate_data(2000)
# X.shape

#### Assignment

In [5]:
# Create the models

assignment_models={}

def nonlinear_transform(X, B, quad_indeces):
    for quad_index in quad_indeces:
        quad = X[:, quad_index]**2
        X = np.hstack([X, quad.reshape(-1, 1)])
        B = np.append(B, B[quad_index])
    
    return X, B

def nonadditive_transform(X, B, interaction_indeces, interaction_weights=None):
    for interaction_index, var_indeces in enumerate(interaction_indeces):
        int_1, int_2 = var_indeces
        interaction_val = X[:, int_1]*X[:, int_2]
        
        if not interaction_weights:
            interaction_val = interaction_val*0.5
        else:
            interaction_val = interaction_val*interaction_weights[interaction_index]
            
        X = np.hstack([X, interaction_val.reshape(-1, 1)])
        B = np.append(B, B[int_1])
    
    return X, B

# Scenario 1
assignment_models["A_add_lin"] = lambda B, X: np.dot(X, B)

# Scenario 2:     
assignment_models["B_add_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2]))
# Scenario 3:
assignment_models["C_add_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2, 4, 7]))
# Scenario 4:
assignment_models["D_mild_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]))

# Scenario 5:
assignment_models["E_mild_nadd_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]), quad_indeces=[2]))
# Scenario 6
assignment_models["F_mod_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]))
# Scenario 7
assignment_models["G_mod_nadd_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]), 
                                                                            quad_indeces=[2, 4, 7]))

In [6]:
# Tests 
assert(set(assignment_models["A_add_lin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([9, 37]))

assert(set(assignment_models["B_add_mild_nlin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([33, 637]))

assert(set(assignment_models["C_add_mod_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([373, 13457]))

assert(set(assignment_models["D_mild_nadd_lin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([139.5, 3632]))

assert(set(assignment_models["E_mild_nadd_mild_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([163.5, 4232]))


#### Outcome and Assignment Functions

In [7]:
def get_assignments(B, X, n_samples, scenario="A_add_lin"):
    X_usable = X[:, treat_vars]
    
    # Calculate the probabilities of assignment
    linear_assignment_data = assignment_models[scenario](B, X_usable)
    p_treat = 1.0/(1+np.exp(-1*linear_assignment_data))

    # Assign
    rand = np.random.random(n_samples)
    assignments = (rand < p_treat).astype(int)
    
    return assignments

In [8]:
def get_outcomes(B, X, assignments, effect=true_treat_effect):
    X_usable = X[:, outcome_vars]
    return effect*assignments + np.dot(X_usable, B)

In [9]:
# DEBUG
# assignments = get_assignments(assignment_weights, X, "mild_nonaddititive_mild_nonlinear")
# outcomes = get_outcomes(outcome_weights, X, assignments)

### Analysis

In [10]:
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats')
matching = importr('Matching')
snow = importr('snow')

Some code is going to >48 hours to ran. Lucky it's highly parellalisable so we can use a compute cluster. The one option is local to split across CPU cores. The better option is to go remote and explote 32 cores on multiple AWS machines.

On AWS, this is straightforward. Manually you need to port forward!. This allows the remote machine to connect to ports on the master via it's localhost loopback. 

```
# ~/.bash_profile
# Allow remote host to connect to local machine
# usage: $ remote_pfwd hostname {6000..6009}
function remote_pfwd {
  for i in ${@:2}
  do
    ssh -N -R $i:localhost:$i $1 &
  done
}
```
`remote_pfwd ubuntu@52.90.20.45 {11305..11307}`

In [11]:
AWS_MASTER_DNS="ip-172-31-42-147.ec2.internal"
AWS_SLAVE_1 = "ubuntu@ip-172-31-43-193.ec2.internal"
AWS_SLAVE_2 = "ubuntu@ip-172-31-81-244.ec2.internal"
AWS_MASTER_PORT_RANGE = list(range(11305, 11340))

class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, ports=None):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            # Set the acceptable ports for connection
            # from the slaves
            if not ports:
                ports = AWS_MASTER_PORT_RANGE
            
            # Construct the connection string
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
                
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(ports), master=AWS_MASTER_DNS, outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [12]:
# Local cluster
# cluster_provider = ClusterProvider(n_nodes=14)

In [12]:
# Remote cluster
cluster_provider = ClusterProvider(remote_hosts=[(AWS_SLAVE_2, 8)],
                                    ports = list(range(11315, 11325)))

In [24]:
# Run this with True to kill the cluster
kill = True
if kill:
    cluster_provider.kill_cluster()

#### Estimators

Define methods which can process outcomes, assignments and covariate data into a treatment effect estimate. 

1. Logistic Regression
2. GenMatch
3. VAE

In [13]:
def get_propensity_scores(assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [14]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data):
    
    propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Run matching
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [15]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Add prop scores to covar data
    propensity_scores = np.array(get_propensity_scores(assignments, covariate_data))
    matching_data = np.hstack([covariate_data, propensity_scores.reshape(-1, 1)])
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=covariate_data,
        print_level=0,
        cluster=cl)
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=matching_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [16]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [17]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

#### Monte Carlo Runner Code

In [18]:
def get_data(n_samples, assignment_model):
    X = generate_data(n_samples)
    assignments = get_assignments(assignment_weights, X,
                                  n_samples, assignment_model)

    outcomes = get_outcomes(outcome_weights, X, assignments)
    
    return assignments, outcomes, X

def get_estimate(outcomes, assignments, covar_data, method, **args):
    return method(outcomes, assignments, covar_data, **args)

In [19]:
def run_simulation(runs=1000, n_samples=1000,
                   assignment_model="additive_linear",
                   estimator=logistic_prop_matching_est,
                   verbose=True,
                   **args):
    
    progress_tick = max(1, int(runs/10))
    results = np.zeros(runs)

    for i in range(runs):
        assignments, outcomes, covar_data = get_data(n_samples, assignment_model)
        
        covar_data = covar_data[:, 1:] #exclude bias term
        results[i] = get_estimate(outcomes, assignments,
                                  covar_data, estimator, **args)
        
        if i%progress_tick == progress_tick-1 and verbose:
            print("Done {} of {}".format(i+1, runs))
    
    bias = np.abs(np.mean((true_treat_effect-results)/true_treat_effect)*100)
    rmse = np.mean((true_treat_effect-results)**2)**0.5
    
    if verbose:
        print("\nRMSE", rmse)
        print("Bias", bias)
        print("===============\n\n")
    
    return {"RMSE": rmse, "Bias": bias}

In [20]:
# DEBUG:
run_simulation(runs=20, n_samples=1000, assignment_model="D_mild_nadd_lin",
              estimator=genmatch_est, verbose=True)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  11.083404302597046
GenMatch Time:  31.12960457801819
Done 2 of 20
GenMatch Time:  11.981449365615845
GenMatch Time:  16.15269660949707
Done 4 of 20
GenMatch Time:  14.55371356010437
GenMatch Time:  13.442266702651978
Done 6 of 20
GenMatch Time:  14.062756061553955
GenMatch Time:  15.857194662094116
Done 8 of 20
GenMatch Time:  8.22305178642273
GenMatch Time:  12.44659686088562
Done 10 of 20
GenMatch Time:  10.008002996444702
GenMatch Time:  13.095482110977173
Done 12 of 20
GenMatch Time:  11.391975164413452
GenMatch Time:  15.019485712051392
Done 14 of 20
GenMatch Time:  10.63561201095581
GenMatch Time:  14.826309204101562
Done 16 of 20
GenMatch Time:  13.209826946258545
GenMatch Time:  12.118821620941162
Done 18 of 20
GenMatch Time:  17.138674020767212
GenMatch Time:  10.279953002929688
Done 20 of 20

RMSE 0.043918225298051665
Bias 0.3755681319660123




{'Bias': 0.3755681319660123, 'RMSE': 0.043918225298051665}

### Run MC trial

In [20]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

def get_store_name(models_being_run, est, runs, n_samples):
    # Storage
    
    if set(models_being_run) == set(assignment_model_names):
        store_name = "est_{}_runs_{}_n_{}".format(
            est.__name__,
            runs,
            n_samples)
    else:
        store_name = "est_{}_runs_{}_n_{}_models_{}".format(
            est.__name__,
            runs,
            n_samples,
            "_".join(models_being_run))
    
    return store_name

def run_test_battery(est,
                     store_name=None, 
                     runs=1000,
                     n_samples=1000,
                     models=assignment_models,
                     overwrite=False, verbosity=1,
                     **args):
    # Logging
    def printer(level, *args):
        if level <= verbosity:
            print(*args)
    
    # Storage config
    if store_name is None:
        store_name = get_store_name(models, est, runs, n_samples)
            
    results = retrieve_results_dict(store_name)

    if overwrite or (not results):
        printer(1, "No valid, existant results found. Beggining battery.\n")
        results = {}
        for model in models:
            printer(1, "Running: ", model)
            results[model] = run_simulation(
                                runs=runs,
                                n_samples=n_samples,
                                assignment_model=model,
                                estimator=est,
                                verbose=(verbosity==2),
                                **args)
            store_results_dict(results[model], store_name+"_checkpoint_"+model)
            printer(1, "Done.\n")

        store_results_dict(results, store_name)
    else:
        printer(1, "Displaying cached results.\n")
    
    printer(1, "Results")
    for model, results in results.items():
        printer(1, "Model: ", model)
        print(1, "Bias: ", results["Bias"])
        print(1, "RMSE: ", results["RMSE"], "\n")

### Run the Logistic Regression Battery

This one is easy. So we run on one machine.

In [21]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=1000,
    n_samples=1000)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.045874914703647685
1 RMSE:  0.07310500057973227 

Model:  B_add_mild_nlin
1 Bias:  3.1844355433209786
1 RMSE:  0.06588422028138122 

Model:  C_add_mod_nlin
1 Bias:  10.094350684204597
1 RMSE:  0.07650839711310455 

Model:  D_mild_nadd_lin
1 Bias:  6.720731771408928
1 RMSE:  0.08531717119502563 

Model:  E_mild_nadd_mild_nlin
1 Bias:  10.36168716658826
1 RMSE:  0.09094245826533698 

Model:  F_mod_nadd_lin
1 Bias:  3.1228082403965436
1 RMSE:  0.07605107262377982 

Model:  G_mod_nadd_mod_nlin
1 Bias:  11.830178367664905
1 RMSE:  0.07798212919046259 



### Run the GenMatch Battery

We split this across three machines using remote clusters.

In [22]:
gm_est = genmatch_est
gm_runs = 1000
gm_n_samples = 1000
gm_models_sets = [assignment_model_names[:3], assignment_model_names[3:5], assignment_model_names[5:]]
gm_files_to_be_produced = []

for model_set in gm_models_sets:
    gm_files_to_be_produced.append(get_store_name(model_set, gm_est, gm_runs, gm_n_samples))

gm_files_to_be_produced

['est_genmatch_est_runs_1000_n_1000_models_A_add_lin_B_add_mild_nlin_C_add_mod_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_D_mild_nadd_lin_E_mild_nadd_mild_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_F_mod_nadd_lin_G_mod_nadd_mod_nlin']

In [26]:
# A-C
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[0],
    verbosity=2)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  20.023235988165702
1 RMSE:  0.08009294395266281 

Model:  B_add_mild_nlin
1 Bias:  13.717727327601459
1 RMSE:  0.054870909310405835 

Model:  C_add_mod_nlin
1 Bias:  20.087548601352452
1 RMSE:  0.0803501944054098 



In [27]:
# D-E
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[1],
    verbosity=2)

Displaying cached results.

Results
Model:  D_mild_nadd_lin
1 Bias:  13.127238676118091
1 RMSE:  0.052508954704472366 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.4125520199797545
1 RMSE:  0.017650208079919016 



In [23]:
# F-G
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[2],
    verbosity=2)

No valid, existant results found. Beggining battery.

Running:  F_mod_nadd_lin


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  9.572449684143066
GenMatch Time:  16.340958833694458
GenMatch Time:  14.121562480926514
GenMatch Time:  13.51052737236023
GenMatch Time:  9.34848952293396
GenMatch Time:  10.77765703201294
GenMatch Time:  18.568121910095215
GenMatch Time:  19.216779947280884
GenMatch Time:  19.958328008651733
GenMatch Time:  6.411807298660278
GenMatch Time:  13.899039268493652
GenMatch Time:  13.144937515258789
GenMatch Time:  23.009070873260498
GenMatch Time:  6.295571565628052
GenMatch Time:  17.436700344085693
GenMatch Time:  10.700563669204712
GenMatch Time:  7.987912178039551
GenMatch Time:  19.093489170074463
GenMatch Time:  19.41653537750244
GenMatch Time:  10.098606824874878
GenMatch Time:  10.701445817947388
GenMatch Time:  23.086807250976562
GenMatch Time:  11.253440856933594
GenMatch Time:  13.568389177322388
GenMatch Time:  25.83503246307373
GenMatch Time:  15.703532934188843
GenMatch Time:  15.628454685211182
GenMatch Time:  12.876771450042725
GenMatch Time:  8.778067588806

GenMatch Time:  12.344777584075928
GenMatch Time:  22.590076684951782
GenMatch Time:  12.758987426757812
GenMatch Time:  25.519463539123535
GenMatch Time:  12.397320985794067
GenMatch Time:  8.757304906845093
GenMatch Time:  12.076944828033447
GenMatch Time:  18.627933979034424
GenMatch Time:  22.480214834213257
GenMatch Time:  14.23400354385376
GenMatch Time:  16.794836282730103
GenMatch Time:  11.370543241500854
GenMatch Time:  8.143033266067505
GenMatch Time:  16.910833835601807
GenMatch Time:  25.044013023376465
GenMatch Time:  9.86852240562439
GenMatch Time:  9.860907793045044
GenMatch Time:  15.866350173950195
GenMatch Time:  13.047085285186768
GenMatch Time:  12.317259550094604
GenMatch Time:  17.41148066520691
GenMatch Time:  10.967252969741821
GenMatch Time:  7.101423025131226
GenMatch Time:  7.9935142993927
GenMatch Time:  19.599998950958252
GenMatch Time:  17.310301303863525
GenMatch Time:  19.561981439590454
GenMatch Time:  10.046629667282104
GenMatch Time:  11.417980194091

GenMatch Time:  15.291515111923218
GenMatch Time:  7.548030853271484
GenMatch Time:  18.53257417678833
GenMatch Time:  10.27302885055542
GenMatch Time:  11.144025564193726
GenMatch Time:  9.008819341659546
GenMatch Time:  10.360104322433472
GenMatch Time:  12.707260847091675
GenMatch Time:  22.159782886505127
GenMatch Time:  7.3720808029174805
GenMatch Time:  8.779756546020508
GenMatch Time:  12.478221893310547
GenMatch Time:  19.24811029434204
GenMatch Time:  9.413668394088745
GenMatch Time:  19.352883338928223
GenMatch Time:  10.647576570510864
GenMatch Time:  17.63392996788025
GenMatch Time:  10.682462215423584
GenMatch Time:  6.960781097412109
GenMatch Time:  15.790157556533813
GenMatch Time:  16.625917673110962
GenMatch Time:  8.772942066192627
GenMatch Time:  14.033875703811646
GenMatch Time:  13.611942768096924
GenMatch Time:  13.228506088256836
GenMatch Time:  18.413621425628662
Done 500 of 1000
GenMatch Time:  14.701188087463379
GenMatch Time:  12.800004005432129
GenMatch Time

...

GenMatch Time:  13.153965711593628
GenMatch Time:  14.826155185699463
GenMatch Time:  9.5063636302948
GenMatch Time:  9.549074172973633
GenMatch Time:  17.66340160369873
GenMatch Time:  10.655338287353516
GenMatch Time:  9.856151103973389
GenMatch Time:  12.733408212661743
GenMatch Time:  16.279143571853638
GenMatch Time:  13.010769367218018
GenMatch Time:  14.844391107559204
GenMatch Time:  16.175126314163208
GenMatch Time:  9.14410400390625
GenMatch Time:  11.591928720474243
GenMatch Time:  15.09946322441101
GenMatch Time:  22.7050564289093
GenMatch Time:  13.47621750831604
GenMatch Time:  11.21258020401001
GenMatch Time:  14.086723566055298
GenMatch Time:  15.516412258148193
GenMatch Time:  12.463999271392822
GenMatch Time:  12.783108711242676
GenMatch Time:  10.390624284744263
GenMatch Time:  8.095049619674683
GenMatch Time:  16.63166832923889
GenMatch Time:  12.637162685394287
GenMatch Time:  13.60312008857727
GenMatch Time:  21.071343898773193
GenMatch Time:  5.54827356338501
Gen

GenMatch Time:  13.395122766494751
GenMatch Time:  8.151979923248291
GenMatch Time:  11.488315343856812
GenMatch Time:  6.515831232070923
GenMatch Time:  16.48797583580017
GenMatch Time:  4.864006519317627
GenMatch Time:  16.454169750213623
GenMatch Time:  12.868783950805664
GenMatch Time:  16.142449140548706
GenMatch Time:  10.393752098083496
GenMatch Time:  23.628190279006958
GenMatch Time:  20.995127201080322
GenMatch Time:  21.766544103622437
GenMatch Time:  16.023970127105713
GenMatch Time:  19.038416147232056
GenMatch Time:  15.282208442687988
GenMatch Time:  8.883784055709839
GenMatch Time:  8.849234580993652
GenMatch Time:  10.995632886886597
Done 200 of 1000
GenMatch Time:  29.303958892822266
GenMatch Time:  30.859131813049316
GenMatch Time:  12.412638425827026
GenMatch Time:  12.220401763916016
GenMatch Time:  13.619019269943237
GenMatch Time:  19.264694690704346
GenMatch Time:  8.352947473526001
GenMatch Time:  16.88001275062561
GenMatch Time:  16.253132104873657
GenMatch Ti

GenMatch Time:  25.67581272125244
GenMatch Time:  10.111194133758545
GenMatch Time:  17.67409658432007
GenMatch Time:  10.60723066329956
GenMatch Time:  17.1639187335968
GenMatch Time:  5.262347936630249
GenMatch Time:  14.12841510772705
GenMatch Time:  8.161958932876587
GenMatch Time:  14.941032409667969
GenMatch Time:  16.70998740196228
GenMatch Time:  12.697093725204468
GenMatch Time:  12.63725471496582
GenMatch Time:  12.503231048583984
GenMatch Time:  7.2718212604522705
GenMatch Time:  16.304118871688843
GenMatch Time:  8.935222148895264
GenMatch Time:  20.21468424797058
GenMatch Time:  24.20476269721985
GenMatch Time:  11.528875589370728
GenMatch Time:  18.826112508773804
GenMatch Time:  10.965501070022583
GenMatch Time:  20.5498788356781
GenMatch Time:  8.115040302276611
GenMatch Time:  18.36772131919861
GenMatch Time:  17.635329246520996
GenMatch Time:  7.771590709686279
GenMatch Time:  18.2011399269104
GenMatch Time:  6.945548057556152
GenMatch Time:  15.35066294670105
GenMatc

GenMatch Time:  8.356083869934082
GenMatch Time:  14.231839418411255
GenMatch Time:  16.13124942779541
GenMatch Time:  9.094398498535156
GenMatch Time:  11.09575891494751
GenMatch Time:  11.47903847694397
GenMatch Time:  19.749202728271484
GenMatch Time:  9.83135437965393
GenMatch Time:  10.920089960098267
GenMatch Time:  9.375283002853394
GenMatch Time:  10.434823036193848
GenMatch Time:  12.707038402557373
GenMatch Time:  17.140730619430542
GenMatch Time:  8.376237630844116
GenMatch Time:  14.732096910476685
GenMatch Time:  11.849789142608643
GenMatch Time:  20.681593894958496
GenMatch Time:  14.890042543411255
GenMatch Time:  17.359267473220825
GenMatch Time:  17.292566299438477
GenMatch Time:  15.149251461029053
GenMatch Time:  12.564473628997803
GenMatch Time:  10.966362714767456
GenMatch Time:  8.949526309967041
GenMatch Time:  16.67546796798706
GenMatch Time:  9.532511711120605
GenMatch Time:  10.704146146774292
GenMatch Time:  8.327136754989624
GenMatch Time:  10.45747470855712

GenMatch Time:  13.106127738952637
GenMatch Time:  21.832245111465454
GenMatch Time:  12.137587308883667
GenMatch Time:  15.666825294494629
GenMatch Time:  9.417036533355713
GenMatch Time:  10.190717220306396
GenMatch Time:  28.406808614730835
GenMatch Time:  7.4358038902282715
Done 900 of 1000
GenMatch Time:  9.180058479309082
GenMatch Time:  8.444974422454834
GenMatch Time:  11.020518064498901
GenMatch Time:  19.360953330993652
GenMatch Time:  22.63792133331299
GenMatch Time:  8.967558860778809
GenMatch Time:  7.690192222595215
GenMatch Time:  18.214205741882324
GenMatch Time:  12.797924280166626
GenMatch Time:  16.932700157165527
GenMatch Time:  13.57043170928955
GenMatch Time:  15.023723363876343
GenMatch Time:  6.320762395858765
GenMatch Time:  4.225433588027954
GenMatch Time:  16.749699354171753
GenMatch Time:  11.539777755737305
GenMatch Time:  8.00113558769226
GenMatch Time:  10.65442419052124
GenMatch Time:  17.069302082061768
GenMatch Time:  8.373990297317505
GenMatch Time:  

#### Combine results

In [29]:
results = {}
for file in gm_files_to_be_produced:
    results.update(retrieve_results_dict(file))

results

{'A_add_lin': {'Bias': 20.023235988165702, 'RMSE': 0.08009294395266281},
 'B_add_mild_nlin': {'Bias': 13.717727327601459, 'RMSE': 0.054870909310405835},
 'C_add_mod_nlin': {'Bias': 20.087548601352452, 'RMSE': 0.0803501944054098},
 'D_mild_nadd_lin': {'Bias': 13.127238676118091, 'RMSE': 0.052508954704472366},
 'E_mild_nadd_mild_nlin': {'Bias': 4.4125520199797545,
  'RMSE': 0.017650208079919016},
 'F_mod_nadd_lin': {'Bias': 17.275217121640445, 'RMSE': 0.06910086848656177},
 'G_mod_nadd_mod_nlin': {'Bias': 0.26776452898076564,
  'RMSE': 0.0010710581159230625}}